In [1]:
# commissioning section.  Run once to clear and upload the Micropython binaries
## %esptool erase
# Download bin file from https://micropython.org/download/esp32/
## %esptool esp32 /home/julian/executables/micropythonbins/esp32-idf4-20210202-v1.14.bin

Executing:
  esptool.py --port /dev/ttyUSB0 erase_flash

esptool.py v2.8
Serial port /dev/ttyUSB0
Connecting....
[Press the PRG button now if required]
Detecting chip type... ESP32
Chip is ESP32D0WDQ6 (revision 1)
Features: WiFi, BT, Dual Core, 240MHz, VRef calibration in efuse, Coding Scheme None
Crystal is 40MHz
MAC: 7c:9e:bd:f5:2a:f4
Uploading stub...
Running stub...
Stub running...
Erasing flash (this may take a while)...
Chip erase completed successfully in 3.1s
Hard resetting via RTS pin...


In [ ]:
# Wiring:
#  BNO055 orientation accelerometer to ESP32
#    Configured as Serial device: Vin=PS1=3V3, GND=GND, SDA=RX2, SCL=TX2

#  I2C devices, connect as: scl = Pin15, sda = Pin4
#    PX4AIRSPEEDV1 differential baro   (Code X)
#    SHT31D = Humidity and temperature sensor (Code S)
#    BME280 = Baro, temperature, Humidity (Code M)

#  Trigger device on Pin27
#    Propeller anemometer with divider set to 15 

#  Analog device on analogreadpin=35
#    Used for hot wire anemometer measurements

In [69]:
%serialconnect


Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [70]:
%sendtofile config.txt

pinled           2
deviceletter     @
connection0      JulianS9,beckaaaa,192.168.43.1,9042
dev1             1,rx16,tx17,Z
fanbladedivider  15
analogreadpin    35
analogreadrate   550


Sent 7 lines (183 bytes) to config.txt.


In [64]:
%sendtofile --source BNO055_class.py
%sendtofile --source UDPblackviewphone.py
%sendtofile --source utils.py
%sendtofile --source bme280.py
%sendtofile --source scani2cdevices.py

%sendtofile --source sopermodule_main.py main.py

Sent 78 lines (2330 bytes) to main.py.


In [71]:
%sendtofile --source scani2cdevices.py
%sendtofile --source sopermodule_main.py main.py

Sent 127 lines (4089 bytes) to main.py.


In [67]:
%serialconnect
%sendtofile --source sopermodule_main.py main.py

Sent 105 lines (3219 bytes) to main.py.
